In [10]:
import json
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

input_file = 'D:\\Blockchain-IOT\\Program01\\data.json'
encrypted_file = 'D:\\Blockchain-IOT\\Program02\\encrypted_data.json'
decrypted_file = 'D:\\Blockchain-IOT\\Program02\\decrypted_data.json'

def read_file(input_file):
    with open(input_file, 'r') as file:
        return json.load(file)

def write_file(output_file, input_data):
    with open(output_file, 'w') as file:
        json.dump(input_data, file, indent=4)

def pad(data):
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(data) + padder.finalize()
    return padded_data

def unpad(data):
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    unpadded_data = unpadder.update(data) + unpadder.finalize()
    return unpadded_data

def encrypt_data(input_file):
    data = read_file(input_file)
    json_data = json.dumps(data).encode('utf-8')
    key = os.urandom(32)
    init_vector = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(init_vector))
    encryptor = cipher.encryptor()
    padded_data = pad(json_data)
    cyphertext = encryptor.update(padded_data) + encryptor.finalize()
    
    encrypted_data = {
        'cyphertext': cyphertext.hex(),
        'key': key.hex(),
        'init_vector': init_vector.hex()
    }
    write_file(encrypted_file, encrypted_data)
    print("Data encrypted and saved to file.")

def decrypt_data(encrypted_file):
    encrypted_data = read_file(encrypted_file)
    cyphertext = bytes.fromhex(encrypted_data['cyphertext'])
    key = bytes.fromhex(encrypted_data['key'])
    init_vector = bytes.fromhex(encrypted_data['init_vector'])
    
    cipher = Cipher(algorithms.AES(key), modes.CBC(init_vector))
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(cyphertext) + decryptor.finalize()
    json_data = unpad(padded_data)
    data = json.loads(json_data.decode('utf-8'))
    
    write_file(decrypted_file, data)
    print("Data decrypted and saved to file.")

encrypt_data(input_file)

decrypt_data(encrypted_file)


Data encrypted and saved to file.
Data decrypted and saved to file.
